In [5]:
import torch
import torch.nn as nn
import torchvision.models as model
import torchvision.transforms as transforms
from PIL import Image

In [2]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [13]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)


Files already downloaded and verified


In [9]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.max_pool2d(x, (2, 2))
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [10]:
net = LeNet()

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [14]:
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 2.257
[1,  4000] loss: 1.946
[1,  6000] loss: 1.729
[1,  8000] loss: 1.617
[1, 10000] loss: 1.551
[1, 12000] loss: 1.495
[2,  2000] loss: 1.438
[2,  4000] loss: 1.379
[2,  6000] loss: 1.377
[2,  8000] loss: 1.349
[2, 10000] loss: 1.320
[2, 12000] loss: 1.316
[3,  2000] loss: 1.259
[3,  4000] loss: 1.223
[3,  6000] loss: 1.241
[3,  8000] loss: 1.238
[3, 10000] loss: 1.225
[3, 12000] loss: 1.207
[4,  2000] loss: 1.137
[4,  4000] loss: 1.146
[4,  6000] loss: 1.174
[4,  8000] loss: 1.127
[4, 10000] loss: 1.127
[4, 12000] loss: 1.142
[5,  2000] loss: 1.066
[5,  4000] loss: 1.062
[5,  6000] loss: 1.069
[5,  8000] loss: 1.087
[5, 10000] loss: 1.066
[5, 12000] loss: 1.082


In [15]:
torch.save(net.state_dict(), 'lenet.pth')

In [16]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

Files already downloaded and verified


In [17]:
net = LeNet()

net.load_state_dict(torch.load('lenet.pth'))

<All keys matched successfully>

In [18]:
def test_network(net, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

In [19]:
test_network(net, test_loader)

Accuracy of the network on the 10000 test images: 59 %


In [21]:
image = Image.open('/content/download.jpg')
image = transform(image).unsqueeze(0)

# Test the image
output = net(image)
_, predicted = torch.max(output, 1)
print('Predicted class:', predicted.item())

Predicted class: 8


#Part 2

In [22]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [23]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [27]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)


Files already downloaded and verified


In [28]:
net = AlexNet()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        # print('1')
        optimizer.step()

        # print('2')
        running_loss += loss.item()
        if i % 30 == 29:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        # print('3')


[1,    30] loss: 0.035
[1,    60] loss: 0.034
[1,    90] loss: 0.035
[1,   120] loss: 0.035
[1,   150] loss: 0.035
[1,   180] loss: 0.035
[1,   210] loss: 0.034
[1,   240] loss: 0.035
[1,   270] loss: 0.035
[1,   300] loss: 0.034
[1,   330] loss: 0.035
[1,   360] loss: 0.035
[1,   390] loss: 0.035
[1,   420] loss: 0.034
[1,   450] loss: 0.035
[1,   480] loss: 0.035
[1,   510] loss: 0.034
[1,   540] loss: 0.035
[1,   570] loss: 0.035
[1,   600] loss: 0.035
[1,   630] loss: 0.035
[1,   660] loss: 0.035
[1,   690] loss: 0.035
[1,   720] loss: 0.035
[1,   750] loss: 0.035
[1,   780] loss: 0.034
[1,   810] loss: 0.034
[1,   840] loss: 0.035
[1,   870] loss: 0.034
[1,   900] loss: 0.034
[1,   930] loss: 0.034
[1,   960] loss: 0.034
[1,   990] loss: 0.034
[1,  1020] loss: 0.034
[1,  1050] loss: 0.034
[1,  1080] loss: 0.034
[1,  1110] loss: 0.034
[1,  1140] loss: 0.034
[1,  1170] loss: 0.034
[1,  1200] loss: 0.034
[1,  1230] loss: 0.034
[1,  1260] loss: 0.034
[1,  1290] loss: 0.034
[1,  1320] 

In [ ]:
torch.save(net.state_dict(), 'alexnet.pth')